In [44]:
from zipfile import error

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [45]:
df = pd.read_csv('regular.csv')
df.describe()

,Unnamed: 0,target_character,branch_factor,total_iterations,n_self_alignment,total_timesteps,seed,Evaluation/llm_iteration,Evaluation/acc_imp_perc,Evaluation/exist_imp_perc,Evaluation/reach_imp_perc,Evaluation/path_length,Evaluation/fn_imp_perc,Evaluation/fp_imp_perc,Evaluation/tn_imp_perc,Evaluation/tp_imp_perc,Evaluation/solvability,Evaluation/playability
count,1734.000000,1734.000000,1734.0,1734.000000,1734.000000,1734.0,1734.000000,1734.000000,1734.000000,1734.000000,1734.000000,1734.000000,1734.000000,1734.000000,1734.000000,1734.000000,1734.000000,1734.000000
mean,889.000000,1.972318,2.0,8.408304,2.168397,50000000.0,3.522491,4.084198,0.211406,0.685909,0.484230,20.511747,2.222530,0.143253,0.090504,0.543714,0.303729,0.578124
std,507.217936,0.823301,0.0,1.194071,2.478625,0.0,1.886053,2.333641,0.321114,0.288846,0.357135,11.125634,1.033412,0.333812,0.259088,0.903214,0.375911,0.437080
min,15.000000,1.000000,2.0,6.000000,0.000000,50000000.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,448.250000,1.000000,2.0,9.000000,0.000000,50000000.0,2.000000,2.000000,0.000000,0.516667,0.133333,26.000000,1.566667,0.000000,0.000000,0.000000,0.000000,0.033333
50%,889.000000,2.000000,2.0,9.000000,0.000000,50000000.0,3.000000,4.000000,0.022222,0.716667,0.455556,26.000002,2.800000,0.000000,0.000000,0.033333,0.100000,0.833333
75%,1329.750000,3.000000,2.0,9.000000,5.000000,50000000.0,5.000000,6.000000,0.333333,0.966667,0.850000,26.370371,3.000000,0.133333,0.000000,0.725000,0.633333,1.000000
max,1763.000000,3.000000,2.0,9.000000,5.000000,50000000.0,8.000000,9.000000,1.000000,1.000000,1.000000,37.600002,3.000000,2.000000,2.000000,3.000000,1.000000,1.000000


In [46]:
df = df[df['pe'] == 'cot']
df = df[df['Evaluation/llm_iteration'].isin([1, 6])]
df.groupby(['pe', 'n_self_alignment', 'target_character', 'Evaluation/llm_iteration'])['Evaluation/acc_imp_perc'].count()

pe   n_self_alignment  target_character  Evaluation/llm_iteration
cot  0                 1                 1                           10
                                         6                            8
                       2                 1                           10
                                         6                            8
                       3                 1                           10
                                         6                            8
     5                 1                 1                           20
                                         6                            6
                       2                 1                           20
                                         6                            8
                       3                 1                           20
                                         6                           10
Name: Evaluation/acc_imp_perc, dtype: int64

In [47]:
base_df = df[(df['n_self_alignment'] == 0) & (df['Evaluation/llm_iteration'] == 1)]
base_df = base_df.groupby(['target_character']).agg({'Evaluation/acc_imp_perc': ['mean']})
base_df.columns = ['Base']
base_df

,Base
target_character,
1,0.017778
2,0.002222
3,0.102222


In [48]:
sa_df = df[(df['n_self_alignment'] == 5) & (df['Evaluation/llm_iteration'] == 1)]
sa_df = sa_df.groupby(['target_character']).agg({'Evaluation/acc_imp_perc': ['mean']})
sa_df.columns = ['+ Self-Alignment']
sa_df

,+ Self-Alignment
target_character,
1,0.006667
2,0.002222
3,0.247778


In [49]:
fb_df = df[(df['n_self_alignment'] == 0) & (df['Evaluation/llm_iteration'] == 6)]
fb_df = fb_df.groupby(['target_character']).agg({'Evaluation/acc_imp_perc': ['mean']})
fb_df.columns = ['+ Feedback']
fb_df

,+ Feedback
target_character,
1,0.002778
2,0.241667
3,0.075000


In [50]:
safb_df = df[(df['n_self_alignment'] == 5) & (df['Evaluation/llm_iteration'] == 6)]
safb_df = safb_df.groupby(['target_character']).agg({'Evaluation/acc_imp_perc': ['mean']})
safb_df.columns = ['+ Feedback & Self-Alignment']
safb_df

,+ Feedback & Self-Alignment
target_character,
1,0.155556
2,0.175000
3,0.011111


In [51]:
merged = pd.merge(base_df, sa_df, on='target_character').merge(fb_df, on='target_character').merge(safb_df, on='target_character')
merged = merged.reset_index()
merged.columns = ['Scenario', 'Base', '+ Self-Alignment', '+ Feedback', '+ Feedback & Self-Alignment']
merged

,Scenario,Base,+ Self-Alignment,+ Feedback,+ Feedback & Self-Alignment
0,1,0.017778,0.006667,0.002778,0.155556
1,2,0.002222,0.002222,0.241667,0.175000
2,3,0.102222,0.247778,0.075000,0.011111
